## Лабораторна робота 8


1. Виділити кому користувач переводить гроші за допомогою
класу Matcher. Виділити висловлювання користувача, що є підтвердженнями
(наприклад, Yes, that would be great.), за допомогою шаблонів. Використати файл з 7-ї лабораторної роботи (не обов’язково усі
висловлювання). Продемонструвати роботу.
2. Застосувати синтаксичні залежності для визначення намірів.
Використати файл з 7-ї лабораторної роботи (не обов’язково усі
висловлювання).

##### 1. Виділення інформації за допомогою класу Matcher

In [1]:
import json
import spacy
from spacy.matcher import Matcher

In [2]:
# Ініціалізація spaCy
nlp = spacy.load("en_core_web_sm")

In [4]:
# Завантаження JSON файлу
with open('../lab7/banks.json', 'r') as file:
    data = json.load(file)

In [6]:
data[0]

{'dialogue_id': '40_00000',
 'services': ['Banks_1'],
 'turns': [{'frames': [{'actions': [{'act': 'INFORM',
       'canonical_values': ['savings'],
       'slot': 'account_type',
       'values': ['savings']},
      {'act': 'INFORM_INTENT',
       'canonical_values': ['TransferMoney'],
       'slot': 'intent',
       'values': ['TransferMoney']}],
     'service': 'Banks_1',
     'slots': [],
     'state': {'active_intent': 'TransferMoney',
      'requested_slots': [],
      'slot_values': {'account_type': ['savings']}}}],
   'speaker': 'USER',
   'utterance': 'Can you assist me in transferring funds from my savings account?'},
  {'frames': [{'actions': [{'act': 'REQUEST',
       'canonical_values': [],
       'slot': 'recipient_account_name',
       'values': []}],
     'service': 'Banks_1',
     'slots': []}],
   'speaker': 'SYSTEM',
   'utterance': 'Yes I can help with that. Who will you be sending the funds to?'},
  {'frames': [{'actions': [{'act': 'INFORM',
       'canonical_values

In [21]:
# Функція для аналізу діалогу
def analyze_dialogue(dialogue):
    matcher = Matcher(nlp.vocab)
    
    # Шаблон для виявлення одержувача переказу
    # to + можливий артикль + власний іменник
    recipient_pattern = [{"LOWER": "to"}, {"POS": "DET", "OP": "?"}, {"POS": "PROPN"}]
    matcher.add("TransferRecipient", [recipient_pattern])
    
    # Шаблони для підтверджувальних висловлювань
    confirmation_patterns = [
        [{"LOWER": {"IN": ["yes", "yeah", "sure", "definitely", "absolutely"]}}, {"IS_PUNCT": True, "OP": "?"}],
        [{"LOWER": "that"}, {"LOWER": "would"}, {"LOWER": "be"}, {"LOWER": "great"}]
    ]
    matcher.add("Confirmation", confirmation_patterns)
    
    for turn in dialogue['turns']:
        if turn['speaker'] == "USER":
            utterance = turn['utterance']
            doc = nlp(utterance)
            
            matches = matcher(doc)
            for match_id, start, end in matches:
                span = doc[start:end]
                match_label = nlp.vocab.strings[match_id]
                print(f"Match ({match_label}): {span.text}")
            


In [22]:
# Аналіз діалогів
for dialogue in data[:50]:
    print(f"Dialogue ID: {dialogue['dialogue_id']}")
    analyze_dialogue(dialogue)

Dialogue ID: 40_00000
Match (Confirmation): Yes
Dialogue ID: 40_00001
Match (TransferRecipient): to Pranav
Match (Confirmation): Yes
Match (Confirmation): Yes.
Dialogue ID: 40_00002
Match (TransferRecipient): to Mr
Match (Confirmation): Yes
Match (Confirmation): Yes,
Dialogue ID: 40_00003
Match (TransferRecipient): to Maria
Match (Confirmation): Yes
Match (Confirmation): Yes,
Match (Confirmation): that would be great
Dialogue ID: 40_00004
Match (TransferRecipient): to Maria
Match (Confirmation): Yes
Match (Confirmation): Yes,
Dialogue ID: 40_00005
Match (TransferRecipient): to Srinivas
Dialogue ID: 40_00006
Match (TransferRecipient): to Abhinav
Dialogue ID: 40_00007
Match (TransferRecipient): to Amir
Match (Confirmation): Yes
Match (Confirmation): Yes,
Dialogue ID: 40_00008
Match (TransferRecipient): to Amir
Match (TransferRecipient): to Raghav
Match (Confirmation): Yes
Dialogue ID: 40_00009
Match (Confirmation): Yes
Match (Confirmation): Yes,
Dialogue ID: 40_00010
Match (TransferRecip

##### 2. Застосування синтаксичних залежностей для визначення намірів

In [15]:
# Завантаження даних з файлу banks.json
with open('../lab7/banks.json', 'r') as f:
    dialogues = json.load(f)

In [25]:

def analyze_intents(dialogue):
    for turn in dialogue['turns']:
        if turn['speaker'] == "USER":
            utterance = turn['utterance']
            doc = nlp(utterance)
            
            print(f"User Utterance: {utterance}")
            
            # Визначення головного дієслова (ROOT) і об'єктів 
            for token in doc:
                if token.dep_ == "ROOT":
                    verb = token.text
                    objects = []
                    
                    for child in token.children:
                        if child.dep_ in ("dobj", "iobj", "pobj"):
                            objects.append(child.text)
                        
                        # Додавання додаткових залежностей для аналізу
                        if child.dep_ == "prep":
                            objects.extend([subchild.text for subchild in child.children if subchild.dep_ == "pobj"])
                    
                    # Вивід результатів
                    print(f"Intent Verb (ROOT): {verb}")
                    if objects:
                        print(f"Objects: {', '.join(objects)}")
                    else:
                        print("No objects found.")
                    
            print("-" * 40)
                    

In [26]:
# Аналіз усіх діалогів
for dialogue in dialogues[50:75]:
    analyze_intents(dialogue)


User Utterance: I would like to transfer to Amir
Intent Verb (ROOT): like
No objects found.
----------------------------------------
User Utterance: One thousand two hundred and forty bucks, with my checking account
Intent Verb (ROOT): bucks
Objects: account
----------------------------------------
User Utterance: Yes, I confirm.
Intent Verb (ROOT): confirm
No objects found.
----------------------------------------
User Utterance: Thank you. Now I would like to check my checking account balance
Intent Verb (ROOT): Thank
Objects: you
Intent Verb (ROOT): like
No objects found.
----------------------------------------
User Utterance: Thank you! Bye
Intent Verb (ROOT): Thank
Objects: you
Intent Verb (ROOT): Bye
No objects found.
----------------------------------------
User Utterance: Can you help me make a transfer please?
Intent Verb (ROOT): help
No objects found.
----------------------------------------
User Utterance: Send 1,770 dollars from my savings account to Xiaoxue please
Intent 